In [1]:
# Sprawdzenie dostępności GPU i CUDA
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Włącz GPU dla TensorFlow
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"CUDA jest dostępna! Używam GPU: {gpus[0].name}")
    except RuntimeError as e:
        print(f"Wystąpił problem z konfiguracją GPU: {e}")
else:
    print("CUDA jest niedostępna. Trening będzie przeprowadzony na CPU.")


CUDA jest niedostępna. Trening będzie przeprowadzony na CPU.


In [2]:
# Wczytywanie ścieżek z config.json
import json
from pathlib import Path


config_path = Path("../config.json")
with open(config_path, "r") as config_file:
    config = json.load(config_file)

train_file = Path(config["train_file"])
dataset_path = Path(config["dataset_path"])

print(f"Wczytano ścieżki:\nTrain File: {train_file}\nDataset Path: {dataset_path}")


Wczytano ścieżki:
Train File: C:\studia\wejherowo_tree\fashion\data\styles_train.csv
Dataset Path: C:\Users\tthaddey\.cache\kagglehub\datasets\paramaggarwal\fashion-product-images-small\versions\1


In [3]:
# Wczytywanie zbioru treningowego
import pandas as pd

df_train = pd.read_csv(train_file)

# Tworzenie pełnych ścieżek do obrazów
# Konwersja każdej ścieżki na obiekt Path
df_train["image_path"] = df_train["id"].apply(lambda x: dataset_path / "images" / f"{x}.jpg")


# Sprawdzenie danych
print("Pierwsze wiersze zbioru treningowego:")
print(df_train.head())
print(f"Łączna liczba próbek: {len(df_train)}")


Pierwsze wiersze zbioru treningowego:
      id gender masterCategory subCategory articleType baseColour  season  \
0  58770    men        apparel     topwear      shirts       grey  summer   
1  31286    men        apparel     topwear     tshirts      white  summer   
2  27277  women        apparel     topwear      kurtas      green    fall   
3  13251    men        apparel  bottomwear       other      beige    fall   
4  36253    men        apparel       other       other  navy blue  summer   

     year   usage                            productDisplayName  \
0  2012.0  casual                     locomotive men grey shirt   
1  2012.0  casual                        puma men white t-shirt   
2  2011.0  ethnic                 mother earth women blue kurta   
3  2011.0  casual       palm tree kids boy solid beige trousers   
4  2016.0  casual  sdl by sweet dreams men navy blue pyjama set   

                                          image_path  
0  C:\Users\tthaddey\.cache\kagglehub\dat

In [4]:
# Ustawienia dla generatora danych
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (224, 224)  # Rozmiar obrazu zgodny z wejściem ResNet18
batch_size = 32

# Funkcja pomocnicza do ładowania obrazów
def preprocess_image(image_path, label):
    # Wczytanie i przeskalowanie obrazu
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = image / 255.0  # Normalizacja do zakresu [0, 1]
    return image, label

# Przygotowanie etykiet
categories = df_train["masterCategory"].unique()
category_to_index = {category: idx for idx, category in enumerate(categories)}
df_train["category_index"] = df_train["masterCategory"].map(category_to_index)

# Konwersja do TensorFlow Dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (df_train["image_path"].astype(str), df_train["category_index"])
)
train_ds = train_ds.map(preprocess_image).shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [5]:
# Wczytanie pretrenowanego modelu ResNet50
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models


base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Zamrożenie warstw

# Dodanie warstw własnych
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(len(categories), activation="softmax")  # Liczba klas zgodna z masterCategory
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,114,055 (91.99 MB)

 Trainable params: 526,343 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
from pathlib import Path

# Sprawdzenie folderu zapisu modelu
models_dir = Path("../models")
models_dir.mkdir(parents=True, exist_ok=True)  # Tworzy katalog, jeśli nie istnieje

# Trening modelu
epochs = 10

history = model.fit(
    train_ds,
    epochs=epochs,
    verbose=1
)


In [ ]:
# Zapis modelu do folderu ../models/
model_save_path = models_dir / "resnet18_masterCategory_model.h5"
model.save(model_save_path)
print(f"Model został zapisany w: {model_save_path}")

# Aktualizacja pliku config.json
config_path = Path("../config.json")
if config_path.exists():
    with open(config_path, "r") as config_file:
        config = json.load(config_file)
else:
    config = {}

config["resnet18_masterCategory_model"] = str(model_save_path.resolve())

with open(config_path, "w") as config_file:
    json.dump(config, config_file, indent=4)

print(f"Plik konfiguracyjny zaktualizowany: {config_path}")